In [1]:
from agentgenius.agents import BaseAgent
from pydantic_ai import RunContext, Tool
from dotenv import load_dotenv
import nest_asyncio
from agentgenius.tools import ToolSet
from datetime import date

nest_asyncio.apply()
load_dotenv()

True

In [2]:
def test_tool(value: int) -> str:
    return str(value)


tools = ToolSet(test_tool)

tool = tools.get("test_tool")
assert tool(10) == "10"

assert "test_tool2" not in tools
assert "test_tool" in tools
assert test_tool in tools

tools.remove("test_tool")
tool = tools.get("test_tool")
assert tool is None

In [3]:
def ask_user_tool(ctx: RunContext[str], question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


tools = ToolSet(ask_user_tool)

agent = BaseAgent(model="openai:gpt-4o-mini", system_prompt="You are a helpful assistant.", toolset=tools)


@agent.system_prompt
def add_the_date() -> str:
    return f"The date is {date.today()}."


print(f"{agent.to_json()=}")

j = agent.to_json()
agent2 = BaseAgent.from_json(j)


print(f"{agent2.to_json()=}")
## Agents tolsets aren't the same, not working in the notebook for some reason

agent.to_json()='{"model": "openai:gpt-4o-mini", "system_prompt": "You are a helpful assistant.", "toolset": ["ask_user_tool"]}'
agent2.to_json()='{"model": "openai:gpt-4o-mini", "system_prompt": "You are a helpful assistant.", "toolset": []}'


In [4]:
result = agent.run_sync("is today my birthday?")
print(result.data)

Could you please provide your birthday so I can determine if today is your birthday?
